In [ ]:
#code for data cleaning

import cv2
from mtcnn import MTCNN
import os
import shutil

# Set the path to the directory containing the images
data_dir = '/content/face/confident'

# Create an instance of the MTCNN model
detector = MTCNN()

# Create a new directory to store the filtered images
filtered_dir = os.path.join(data_dir, 'filtered_conf')
os.makedirs(filtered_dir, exist_ok=True)

# Loop through each image in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Read the image file
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        # Detect faces in the image
        faces = detector.detect_faces(img)
        # Check if the image contains any faces
        if len(faces) == 0:
            # If there are no faces, move the image to the filtered directory
            shutil.move(img_path, os.path.join(filtered_dir, filename))
            print(f"{filename} moved to filtered directory")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG19
import os
import numpy as np
import cv2
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Define paths for train, val and test sets
train_data_dir = '/content/confident-unconfident/train'
validation_data_dir = '/content/confident-unconfident/val'
test_data_dir = '/content/confident-unconfident/test'

# Define image dimensions and batch size
img_height = 256
img_width =256
batch_size = 32
num_classes=2

# Load train, val and test sets

train_datagen = ImageDataGenerator(
    rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3),padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, (3, 3),padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(16, (3, 3), padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Define the optimizer with a specific learning rate
#Optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

# Compile the model
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'])


# Define callbacks
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history = model.fit(train_generator,
                    epochs=5,
                    validation_data=validation_generator,
                    callbacks=[tensorboard_callback, early_stopping_callback])

# to get detail analysis of epochs go to anaconda prompt cd to directory where logs are store and execute 
# following command tensorboard --logdir=. --bind_all
#it will give you url for tensorboard
  
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()


#Save the model
model.save('/content/model.h5')







In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG19
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
resize = tf.image.resize(img, (48,48))

In [ ]:
img_width, img_height = 48,48
batch_size = 32
epochs = 10
num_classes = 2

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  #my inference remove dropout layer because may be my model required more feature
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
train_data_dir = '/content/confident-unconfident/train'
validation_data_dir = '/content/confident-unconfident/val'
test_data_dir = '/content/confident-unconfident/test'

train_datagen = ImageDataGenerator(rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


In [ ]:

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//batch_size)

In [ ]:
#Save the model
model.save('/content/modelvgg19.h5')

In [ ]:
#to plot evaluation matrix

# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define test data directory and image dimensions
test_data_dir = '/content/confident-unconfident/test'
img_width, img_height = 256, 256  # replace with your image dimensions

# Create an ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the test data from directory using flow_from_directory method
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=1,
        shuffle=False,
        class_mode='binary')

# Load the saved model
model = tf.keras.models.load_model('/content/Confident_level_prediction_final_9_march.h5')

# Make predictions on the test data
y_pred = model.predict(test_generator)
y_pred_classes = np.round(y_pred).astype(int)

# Get the true labels of the test data
y_true = test_generator.classes

# Calculate the evaluation metrics
#print(classification_report(y_true, y_pred_classes))
#print("Confusion Matrix:")
#print(confusion_matrix(y_true, y_pred_classes))

#heat map
#heatmap
# generate classification report and confusion matrix
print(classification_report(y_true, y_pred_classes))
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# create heatmap for the confusion matrix
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d")
plt.title("Confusion Matrix")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.show()


# Calculate the ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
import matplotlib.pyplot as plt
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
#plt.show()


In [ ]:
#testing of model

from tensorflow import keras

model = keras.models.load_model('/content/Confident_level_prediction_final_9_march.h5')
model.compile() 

import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
img = cv2.imread("/content/240_F_288471634_PRcyq2VkvTVlhHBDUkiwBRh0TQSDiTiy.jpg")
# plt.imshow(img)
# plt.show()

resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

yhat = model.predict(np.expand_dims(resize/255, 0))
print(yhat)
if yhat >= 0.55:
    print(f'Predicted class is confident')
elif(yhat<=0.35):
    print(f'Predicted class is unconfident')
else:
    print(f'prediction class is neutral')
